# オペレーティングシステム 演習 3
# 不可分命令を用いた更新

**このセルを編集して, 名前と学生証番号を書け.**

 * 名前 Name : 03-190503
 * 学生証番号 Student ID : 西山　晃人

**書けたら Shift + Enter で実行(入力を確定)させ, 保存(`Ctrl-S`)せよ**

## 1. 不可分命令

* 不可分命令を使って, mutexを使わずに不可分に更新を行う方法
* 使える場面は限定されているが使える場面では使ったほうが性能が良い

* 以下のプログラムを思い出す

In [1]:
//% file: race.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o race race.c

#include <assert.h>
#include <stdio.h>
#include <unistd.h>
#include <omp.h>

int main() {
  volatile long x = 0;
#pragma omp parallel num_threads(2)
  {
    for (long i = 0; i < 10000000; i++) {
      x++;                      /* 競合状態!! */
    }
  }
    
  printf("x = %ld\n", x); fflush(stdout);
  assert(x == 20000000);        /* これは(きっと)エラーになる */
  return 0;
}


In [2]:
//% cmd: ./race

x = 10593403


race: race.c:19: main: Assertion `x == 20000000' failed.
[C kernel] command exited with code -6, subsequent commands will not be executed

* 結果が(ほとんどの場合)正しく 20000000 にはならない理由は以下の命令列が不可分に実行されないことにある
(R はレジスタ. x は x が格納されているアドレス)

```
R = x
R = R + 1
x = R
```

もう少し本物の機械語風に書くと,

```
movl x,%rax
addl $1,%rax
movl %rax,x
```

もちろん必ずこういう命令列になるなどという保証はないが, ここでのポイントは

* CPUは(通常)1 命令ごとの不可分性は保証している(*)が, 複数命令が不可分に実行されることは保証しない
* x++ という命令が1命令では実行できない

ということである. 

* 注: (*)は例えば movl $100,x (x=100) と movl $200,x (x=200) が(ほぼ)同時に起こった時に, 100でも200でもない(たとえば両者のbitが混ざったような)結果が書かれたり, movl $100,x と movl x,%rax が(ほぼ)同時に起こった時に, 100でも, 元あった値でもない結果が読み込まれたりすることはない, ということ.

* 逆に言うとCPUにもともと「あるアドレスを1加算する」という命令が備わっていればそれを使えばよいということになる. 

* 実を言うとそのような命令(fetch-and-add)は多くのCPUに備わっているのだが, 足し算以外にもやりたくなることがあるだろう. どれだけの命令を作ればいいのか切りがないことになる

* 一般的に, read-modify-write (あるアドレスを読み込み, 変更したっけかを書き込む)操作を不可分に実行できる命令として, compare-and-swap という命令がある

* https://gcc.gnu.org/onlinedocs/gcc-4.1.0/gcc/Atomic-Builtins.html の中の

```
bool __sync_bool_compare_and_swap (type *ptr, type oldval type newval, ...)
```

を参照 (以降は cas と書く)

*
```
cas(p, x, x_new)
```
は 
```
if (*p == x) { *p = x_new; return 1; }
else { return 0 }
```
という操作を不可分に行う(つまり, *p を読み出してから *p に x_new を書き込むまでの間に *p が書き換わっていることがないことが保証されている)

## <font color="blue">課題 3-1:</font> compare-and-swapを使った不可分更新

* _アドレスp にある値を不可分に +1 する関数_ `inc_cas(p)` _を compare-and-swap 命令を使って実装せよ_
* _以下のコードが正しく動く(終了時にx == 20000000となる)ようにせよ_

In [21]:
//% file: hello_cas_1.c
//% cmd: gcc -O3 -Wall -Wextra -fopenmp -o hello_cas_1 hello_cas_1.c

/* ------- このセルを修正して解答を書け. write your answer here ------- */

#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <omp.h>

/* *p を不可分に +1 する関数を 
   __sync_bool_compare_and_swap
   を使って実装 */
void inc_cas(volatile long * p) {
  long pre = *p;
  long nt = pre+1;
  while(__sync_bool_compare_and_swap(p, pre, nt) == 0){
      pre = *p;
      nt = pre+1;
  }
}

int main() {
  volatile long x = 0;
#pragma omp parallel num_threads(2)
  {
    for (long i = 0; i < 10000000; i++) {
      inc_cas(&x);
    }
  }
  printf("x = %ld\n", x); fflush(stdout);
  assert(x == 20000000);
  printf("OK\n");
  return 0;
}


In [22]:
//% cmd: ./hello_cas_1
/* ------- このセルを実行して結果を確かめよ Execute this cell and check the result ------- */

x = 20000000
OK
